In [1]:
from q_lab_toolbox.my_functions import get_paulis

In [2]:
pauli_list, pauli_list_names, id_qubit_list, index_list = get_paulis(2, "full")

In [3]:
pauli_list

array([[[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j]],

       [[ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
        [ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j],
        [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j]],

       [[ 0.+0.j,  0.-1.j,  0.+0.j,  0.+0.j],
        [ 0.+1.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j,  0.-1.j],
        [ 0.+0.j,  0.+0.j,  0.+1.j,  0.+0.j]],

       [[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j, -1.+0.j,  0.+0.j, -0.+0.j],
        [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
        [ 0.+0.j, -0.+0.j,  0.+0.j, -1.+0.j]],

       [[ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j],
        [ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j]],

       [[ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j],
        [ 0.+0.j,  0.+0.

In [4]:
pauli_list_names

array([['I', 'I'],
       ['I', 'X'],
       ['I', 'Y'],
       ['I', 'Z'],
       ['X', 'I'],
       ['X', 'X'],
       ['X', 'Y'],
       ['X', 'Z'],
       ['Y', 'I'],
       ['Y', 'X'],
       ['Y', 'Y'],
       ['Y', 'Z'],
       ['Z', 'I'],
       ['Z', 'X'],
       ['Z', 'Y'],
       ['Z', 'Z']], dtype='<U1')

In [5]:
id_qubit_list

array([[0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1.]])

In [6]:
pauli_index_x, pauli_index_y, pauli_index_factor = index_list

In [7]:
pauli_index_x

array([[0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3]], dtype=int64)

In [8]:
pauli_index_y

array([[0, 1, 2, 3],
       [1, 0, 3, 2],
       [1, 0, 3, 2],
       [0, 1, 2, 3],
       [2, 3, 0, 1],
       [3, 2, 1, 0],
       [3, 2, 1, 0],
       [2, 3, 0, 1],
       [2, 3, 0, 1],
       [3, 2, 1, 0],
       [3, 2, 1, 0],
       [2, 3, 0, 1],
       [0, 1, 2, 3],
       [1, 0, 3, 2],
       [1, 0, 3, 2],
       [0, 1, 2, 3]], dtype=int64)

In [9]:
pauli_index_factor

array([[ 1.+0.j,  1.+0.j,  1.+0.j,  1.+0.j],
       [ 1.+0.j,  1.+0.j,  1.+0.j,  1.+0.j],
       [ 0.-1.j,  0.+1.j,  0.-1.j,  0.+1.j],
       [ 1.+0.j, -1.+0.j,  1.+0.j, -1.+0.j],
       [ 1.+0.j,  1.+0.j,  1.+0.j,  1.+0.j],
       [ 1.+0.j,  1.+0.j,  1.+0.j,  1.+0.j],
       [ 0.-1.j,  0.+1.j,  0.-1.j,  0.+1.j],
       [ 1.+0.j, -1.+0.j,  1.+0.j, -1.+0.j],
       [ 0.-1.j,  0.-1.j,  0.+1.j,  0.+1.j],
       [ 0.-1.j,  0.-1.j,  0.+1.j,  0.+1.j],
       [-1.-0.j,  1.+0.j,  1.+0.j, -1.+0.j],
       [ 0.-1.j,  0.+1.j,  0.+1.j, -0.-1.j],
       [ 1.+0.j,  1.+0.j, -1.+0.j, -1.+0.j],
       [ 1.+0.j,  1.+0.j, -1.+0.j, -1.+0.j],
       [ 0.-1.j,  0.+1.j,  0.+1.j, -0.-1.j],
       [ 1.+0.j, -1.+0.j, -1.+0.j,  1.-0.j]])